OpenVINO 준비

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%pip install -q "openvino>=2023.1.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 15.1 MB/s eta 0:00:00


In [3]:
!pip install openvino-dev

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 67.7 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.3
    Uninstalling networkx-3.3:
      Successfully uninstalled networkx-3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-runtime-cu12==12.1.105; platform_system == "

keras 모델을 Intel OpenVINO IR 파일(xml, bin)로 한다.

In [14]:
import openvino
import openvino as ov
from openvino.tools.mo import convert_model

BASE_PATH = './drive/MyDrive/data/'
ov_model = convert_model(BASE_PATH+'bin/mobilenetv2_class')
ov.save_model(ov_model, BASE_PATH+'bin/mobilenetv2_class8.xml')

[ INFO ] MO command line tool is considered as the legacy conversion API as of OpenVINO 2023.2 release.
In 2025.0 MO command line tool and openvino.tools.mo.convert_model() will be removed. Please use OpenVINO Model Converter (OVC) or openvino.convert_model(). OVC represents a lightweight alternative of MO and provides simplified model conversion API. 
Find more information about transition from MO to OVC at https://docs.openvino.ai/2023.2/openvino_docs_OV_Converter_UG_prepare_model_convert_model_MO_OVC_transition.html


### 테스트 파일 목록으로 읽어와서 답안 출력하기.



In [15]:
# --------------------------- Step 1. Initialize OpenVINO Runtime Core ------------------------------------------------
core = ov.Core()

In [16]:
# --------------------------- Step 2. Read a model --------------------------------------------------------------------
# (.xml and .bin files) or (.onnx file)
model = core.read_model(BASE_PATH+'bin/mobilenetv2_class8.xml')

if len(model.inputs) != 1: print('Sample supports only single input topologies')
if len(model.outputs) != 1: print('Sample supports only single output topologies')

In [17]:
import os

directory_path = "./drive/MyDrive/data/val"

# Get a list of all files in the directory
folder_list = [f for f in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, f))]

# Print the list of files
print("List of files in the directory:")
file_list = []
for folder in folder_list:
    flist = [f for f in os.listdir(directory_path+'/'+folder) if os.path.isfile(os.path.join(directory_path+'/'+folder, f))]
    for file in flist:
        file_list.append(directory_path+'/'+folder+"/"+file)
        print(file)

List of files in the directory:
2008_002151.jpg
2008_001160.jpg
2008_001805.jpg
2008_001380.jpg
2008_000805.jpg
2008_001468.jpg
2008_001774.jpg
2008_002358.jpg
2008_002454.jpg
2008_001971.jpg
2008_000251.jpg
2008_001227.jpg
2008_000367.jpg
2008_001742.jpg
2008_002138.jpg
2008_000804.jpg
2008_001546.jpg
2008_000064.jpg
2008_000021.jpg
2008_001985.jpg
2008_007374.jpg
2008_005253.jpg
2008_004030.jpg
2008_007738.jpg
2008_006631.jpg
2008_003423.jpg
2008_004885.jpg
2008_007409.jpg
2008_003189.jpg
2008_003926.jpg
2008_007458.jpg
2008_003856.jpg
2008_005779.jpg
2008_005455.jpg
2008_006216.jpg
2008_007690.jpg
2008_007764.jpg
2008_007618.jpg
2008_002773.jpg
2008_006548.jpg
2008_003876.jpg
2008_006980.jpg
2008_007507.jpg
2008_004433.jpg
2008_006169.jpg
2008_006034.jpg
2008_002673.jpg
2008_006621.jpg
2008_005611.jpg
2008_006269.jpg
2008_006394.jpg
2008_005750.jpg
2008_007350.jpg
2008_005889.jpg
2008_004348.jpg
2008_003673.jpg
2008_007480.jpg
2008_004935.jpg
2008_003905.jpg
2008_004646.jpg
2008_006

In [19]:
import pickle

with open(BASE_PATH+'bin/class8.pickle', 'rb') as f:
    labels = pickle.load(f)

파일 추론하기

In [20]:
import pandas as pd
import cv2
import numpy as np

# --------------------------- Step 4. Loading model to the device -----------------------------------------------------
print('Loading the model to the plugin')
compiled_model = core.compile_model(model, 'CPU')
img_height = 224

result = []

res = open(BASE_PATH+"result.txt", "w")

print('Starting inference in synchronous mode')
cnt = 0;

for file in file_list:
    # --------------------------- Step 5. Set up input --------------------------------------------------------------------
    # Read input image
    image_path = file
    org_image = cv2.imread(image_path)
    image = cv2.resize(org_image, (img_height,img_height), fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)
    #img = cv2.resize(img, (224,224), fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image / 255.

    # Add N dimension
    nchw_tensor = np.expand_dims(image, 0)

    # NHWC -> NCHW
    input_tensor = np.transpose(nchw_tensor, (0,1,2,3))#(0,3,1,2))

    # --------------------------- Step 6. Create infer request and do inference synchronously -----------------------------
    results = compiled_model.infer_new_request({0:input_tensor})

    # --------------------------- Step 7. Process output ------------------------------------------------------------------
    predictions = next(iter(results.values()))
    output_node_name = next(iter(results.keys())) #

    # Change a shape of a numpy.ndarray with results to get another one with one dimension
    probs = predictions.reshape(-1)

    # Get an array of 8 class IDs in descending order of probability
    top_8 = np.argsort(probs)[-8:][::-1]
    print(top_8)

    header = 'class_id            probability'
    print(f'Image path: {image_path}')
    print('Top 8 results: ')
    print(header)
    print('-' * len(header))

    for class_id in top_8:
        probability_indent = ' ' * (len('class_id           ') - len(labels[class_id]) + 1)
        print(f'{labels[class_id]}{probability_indent}{probs[class_id]*100:.7f}')

    res.write(file+':top1 = ('+labels[top_8[0]]+')'+str(probs[top_8[0]]*100)+', top2 = ('+labels[top_8[1]]+')'+str(probs[top_8[1]]*100)+'\n')

res.close()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
bottle              99.9984145
cat                 0.0013422
car                 0.0001848
bird                0.0000285
bicycle             0.0000131
boat                0.0000129
aeroplane           0.0000017
bus                 0.0000003
[4 6 0 1 2 5 3 7]
Image path: ./drive/MyDrive/data/val/bottle/2008_001060.jpg
Top 8 results: 
class_id            probability
-------------------------------
bottle              99.4892359
car                 0.3471196
aeroplane           0.1094101
bicycle             0.0330869
bird                0.0156081
bus                 0.0050358
boat                0.0002676
cat                 0.0002352
[4 7 6 1 2 0 3 5]
Image path: ./drive/MyDrive/data/val/bottle/2008_002775.jpg
Top 8 results: 
class_id            probability
-------------------------------
bottle              99.9999762
cat                 0.0000122
car                 0.0000061
bicycle             0.0000005
bird                0.0000003
aeroplane      